In [ ]:
!pip install -q git+https://github.com/facebookresearch/segment-anything.git
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q datasets
!pip install -q monai
!pip install -q ultralytics

  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.7/877.7 kB 27.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from scipy import ndimage
from DataPreprocess import ImageData, MaskData, BBox, Reshape
from ultralytics import YOLO
import torch

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from helper import Helper

helperfns = Helper()


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
ROOT_DIR = '/content/gdrive/My Drive/coding/'

In [ ]:
folder_path = ROOT_DIR + 'dl_challenge'

image_data = ImageData()
mask_data = MaskData()
resize_data = Reshape(256, 256)

In [ ]:
image_data.read_all_files(folder_path)
mask_data.read_all_files(folder_path)
combined_masks = resize_data.combine_masks(mask_data.masks)
for i in range(len(combined_masks)):
    combined_masks[i] = combined_masks[i].astype(np.uint8)

In [ ]:
from sklearn.model_selection import train_test_split

images_train, images_temp, combined_masks_train, combined_masks_temp, mask_data_train, mask_data_temp = train_test_split(
    image_data.images, combined_masks, mask_data.masks, test_size=0.2, random_state=42
)

images_val, images_test, combined_masks_val, combined_masks_test, mask_data_val, mask_data_test = train_test_split(
    images_temp, combined_masks_temp, mask_data_temp, test_size=0.5, random_state=42
)

# Yolo Evaluation

In [ ]:
model_path = ROOT_DIR + "yolov8_instance_seg/models/best.pt"

model = YOLO(model_path)

In [ ]:
iou_score_list = []
gt_obj_list = []
detected_obj_list = []
for image, gt_mask, obj_mask in zip(images_test, combined_masks_test, mask_data_test):
    masks_present = False
    results = model(image)
    for result in results:
        if result.masks:
            masks_present = True
            detected_obj_list.append(result.masks.data.shape[0])

    if masks_present:
        yolo_seg_mask = helperfns.generate_mask(image, results)
        yolo_combined_mask = helperfns.combine_masks(yolo_seg_mask)
        iou_score = helperfns.calculate_iou(yolo_combined_mask, gt_mask)
        iou_score_list.append(iou_score)
        gt_obj_list.append(obj_mask.shape[0])




0: 576x640 (no detections), 582.8ms
Speed: 30.6ms preprocess, 582.8ms inference, 17.0ms postprocess per image at shape (1, 3, 576, 640)

0: 480x640 2 objects, 266.2ms
Speed: 5.3ms preprocess, 266.2ms inference, 31.1ms postprocess per image at shape (1, 3, 480, 640)

0: 448x640 6 objects, 237.7ms
Speed: 6.9ms preprocess, 237.7ms inference, 24.2ms postprocess per image at shape (1, 3, 448, 640)

0: 512x640 9 objects, 283.7ms
Speed: 4.5ms preprocess, 283.7ms inference, 32.3ms postprocess per image at shape (1, 3, 512, 640)

0: 544x640 3 objects, 308.6ms
Speed: 5.1ms preprocess, 308.6ms inference, 21.4ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 5 objects, 257.5ms
Speed: 5.4ms preprocess, 257.5ms inference, 16.1ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 4 objects, 235.8ms
Speed: 4.7ms preprocess, 235.8ms inference, 14.1ms postprocess per image at shape (1, 3, 544, 640)

0: 416x640 3 objects, 221.1ms
Speed: 6.1ms preprocess, 221.1ms inference, 12.8ms p

In [ ]:
if len(iou_score_list) > 0:
    mean_value = sum(iou_score_list) / len(iou_score_list)
else:
    mean_value = 0
print(f'Mean IoU: {mean_value}')

mean IOU: 0.7138095065090015


In [ ]:
mean_abs_error = np.mean(np.abs(np.array(gt_obj_list) - np.array(detected_obj_list)))
print(f'Mean Absolute Error: {mean_abs_error}')

Mean Absolute Error: 3.888888888888889


# SAM Evaluation

In [ ]:
from ultralytics import YOLO
from segment_anything import sam_model_registry, SamPredictor
model_path = ROOT_DIR + 'yolov8/models/best.pt'
model = YOLO(model_path)


sam_checkpoint = ROOT_DIR + "SAM/models/best.pth"
model_type = "vit_b"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)



In [ ]:
iou_score_list = []
gt_obj_list = []
detected_obj_list = []
for image, gt_mask, obj_mask in zip(images_test, combined_masks_test, mask_data_test):
    results = model(image)
    yolo_boxes = results[0].boxes
    detected_obj_list.append(len(yolo_boxes))
    yolo_boxes_list = yolo_boxes.xyxy.cpu().numpy().tolist()
    predictor.set_image(image)
    input_boxes = torch.tensor(yolo_boxes_list, device=predictor.device)
    transformed_boxes = predictor.transform.apply_boxes_torch(input_boxes, image.shape[:2])
    masks, _, _ = predictor.predict_torch(
        point_coords=None,
        point_labels=None,
        boxes=transformed_boxes,
        multimask_output=False,
    )
    sam_seg_mask = masks.squeeze(1).cpu().numpy()
    sam_combined_mask = helperfns.combine_masks(sam_seg_mask)
    iou_score = helperfns.calculate_iou(sam_combined_mask, gt_mask)
    iou_score_list.append(iou_score)
    gt_obj_list.append(obj_mask.shape[0])



0: 576x640 2 objects, 9.7ms
Speed: 5.0ms preprocess, 9.7ms inference, 1.3ms postprocess per image at shape (1, 3, 576, 640)

0: 480x640 2 objects, 7.9ms
Speed: 3.0ms preprocess, 7.9ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 448x640 12 objects, 7.6ms
Speed: 2.7ms preprocess, 7.6ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)

0: 512x640 14 objects, 7.1ms
Speed: 4.1ms preprocess, 7.1ms inference, 1.1ms postprocess per image at shape (1, 3, 512, 640)

0: 544x640 3 objects, 7.4ms
Speed: 3.2ms preprocess, 7.4ms inference, 1.1ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 17 objects, 11.3ms
Speed: 4.5ms preprocess, 11.3ms inference, 1.8ms postprocess per image at shape (1, 3, 544, 640)

0: 544x640 21 objects, 10.5ms
Speed: 3.3ms preprocess, 10.5ms inference, 1.8ms postprocess per image at shape (1, 3, 544, 640)

0: 416x640 6 objects, 10.3ms
Speed: 2.5ms preprocess, 10.3ms inference, 1.5ms postprocess per image at shape (1, 3, 

In [ ]:
if len(iou_score_list) > 0:
    mean_value = sum(iou_score_list) / len(iou_score_list)
else:
    mean_value = 0
print(f'Mean IoU: {mean_value}')

Mean IoU: 0.8047080937797743


In [ ]:
mean_abs_error = np.mean(np.abs(np.array(gt_obj_list) - np.array(detected_obj_list)))
print(f'Mean Absolute Error: {mean_abs_error}')

Mean Absolute Error: 1.2
